In [2]:
import json
import pandas as pd
import re
import torch

from evaluate import load
from pathlib import Path

2023-06-14 12:58:47.236371: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 12:58:47.864415: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df_mnli = pd.read_json("./data/multinli_1.0/multinli_1.0_dev_matched.jsonl", lines=True)
df_transformations = pd.read_json("./results/multinli_transforms_dev_matched.jsonl", lines=True)
df = df_mnli.join(df_transformations.set_index('pairID'), on='pairID')
df.head()

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,style,sentence1_transform1,sentence1_transform2,sentence2_transform1,sentence2_transform2
0,"[neutral, entailment, neutral, neutral, neutral]",slate,neutral,63735n,63735,The new rights are nice enough,( ( The ( new rights ) ) ( are ( nice enough )...,(ROOT (S (NP (DT The) (JJ new) (NNS rights)) (...,Everyone really likes the newest benefits,( Everyone ( really ( likes ( the ( newest ben...,(ROOT (S (NP (NN Everyone)) (VP (ADVP (RB real...,pir,"Arrr, thar be some fine new rights fer us!","'Tis a pleasant sight, these new rights, me he...","Arrr, me hearties be lovin' the latest booty.",All hands on deck be happy with the brand new ...
1,"[contradiction, contradiction, contradiction, ...",government,contradiction,91383c,91383,This site includes a list of all award winners...,( ( This site ) ( ( includes ( ( ( ( a list ) ...,(ROOT (S (NP (DT This) (NN site)) (VP (VBZ inc...,The Government Executive articles housed on th...,( ( ( The ( Government ( Executive articles ) ...,(ROOT (S (NP (NP (DT The) (NNP Government) (NN...,pir,This site be includin' a list o' all award win...,"Arrr, this here site be havin' a list o' all t...",Ye Government Executive articles housed on tha...,Thar Government Executive pieces housed on th'...
2,"[entailment, entailment, entailment, entailmen...",telephone,entailment,755e,755,uh i don't know i i have mixed emotions about ...,( ( ( ( uh ( i ( ( do n't ) ( know ( ( i i ) (...,(ROOT (SINV (S (S (INTJ (UH uh)) (NP (FW i)) (...,"I like him for the most part, but would still ...",( I ( ( ( ( ( ( like him ) ( for ( the ( most ...,(ROOT (S (NP (PRP I)) (VP (VP (VBP like) (NP (...,pir,"Arrr, I be not sure me hearties, sometimes I f...","Aye, me hearty, me feelings be mixed about him...","I be likin' him for the most part, but I'd sti...","I be fancyin' him for the most part, but I'd s..."
3,"[contradiction, contradiction, contradiction, ...",telephone,contradiction,78013c,78013,yeah i i think my favorite restaurant is alway...,( yeah ( ( i i ) ( think ( ( my ( favorite res...,(ROOT (S (VP (VB yeah) (NP (NP (FW i) (FW i)) ...,My favorite restaurants are always at least a ...,( ( My ( favorite restaurants ) ) ( ( ( ( are ...,(ROOT (S (NP (PRP$ My) (JJ favorite) (NNS rest...,pir,"Aye, me favorite galley be always the one near...","Arr, methinks me top tavern's always been the ...",Me favorite taverns be always at least a hundr...,My preferred grub spots be a hundred leagues o...
4,"[contradiction, contradiction, contradiction, ...",telephone,contradiction,96377c,96377,i don't know um do you do a lot of camping,( i ( ( do n't ) ( know ( um ( do ( you ( do (...,(ROOT (S (NP (FW i)) (VP (VBP do) (RB n't) (VP...,I know exactly.,( I ( ( know exactly ) . ) ),(ROOT (S (NP (PRP I)) (VP (VBP know) (ADVP (RB...,pir,"I don't be knowin' matey, do ye partake in a l...","Arrr, I be clueless, but do yer fancy a lot o'...",I savvy precisely.,I be knowin' precisely.


In [4]:
if torch.cuda.device_count():
    print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 3080


In [5]:
# huggingface evaluate for BERTScore (roberta-large)

bertscore = load("bertscore")

scores = {}
for ref_col in ('sentence1', 'sentence2'):
    references = df[ref_col].tolist()
    for pred_col in ('transform1', 'transform2'):
        predictions = df[f"{ref_col}_{pred_col}"].tolist()
        
        scores[f"{ref_col}_{pred_col}_bertscore"] = bertscore.compute(predictions=predictions, references=references, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
for key, value in scores.items():
    df[key] = value['f1']
df.head()

In [10]:
save_path = Path("./results/multinli_transforms_bertscore_dev_matched.jsonl")
df.to_json(save_path, orient="records", lines=True)